按照Paddle的处理方式进行可视化XFUND。XFUND采用Paddle标注格式

In [51]:
import json
from ppocr.utils.visual import draw_re_results
import numpy as np
import cv2
import os


def trans_poly_to_bbox(poly):
    x1 = int(np.min([p[0] for p in poly]))
    x2 = int(np.max([p[0] for p in poly]))
    y1 = int(np.min([p[1] for p in poly]))
    y2 = int(np.max([p[1] for p in poly]))
    return [x1, y1, x2, y2]


def draw_labels(ocr_infos, image):
    my_relations = []

    ocr_infos = [info for info in ocr_infos if len(info["transcription"]) > 0]

    # class VQATokenLabelEncode
    relations = []
    id_ocr_infos = {}
    for info in ocr_infos:
        info["bbox"] = trans_poly_to_bbox(info["points"])
        id_ocr_infos[info["id"]] = info
        relations.extend([tuple(sorted(l)) for l in info["linking"]])

    relations = list(set(relations))
    relations = sorted(relations)

    result = []
    for head, tail in relations:
        pair = [id_ocr_infos[head]['label'], id_ocr_infos[tail]['label']]
        if pair == ["answer", "question"]:
            head, tail = tail, head
            my_relations.append((head, tail))
        elif pair == ["question", "answer"]:
            my_relations.append((head, tail))
            pass
        else:
            # print("Error: ", pair)
            pass
        ocr_info_head = id_ocr_infos[head]
        ocr_info_tail = id_ocr_infos[tail]
        result.append((ocr_info_head, ocr_info_tail))

    canvas = draw_re_results(image, result)
    from PIL import Image

    my_relations = sorted(list(set(my_relations)))
    from collections import defaultdict
    groups = defaultdict(list)
    for head, tail in my_relations:
        groups[tail].append(head)

    for tail, heads in groups.items():
        if len(heads) > 1:
            print(tail, heads)

    return Image.fromarray(canvas)


label_file = "train_data/XFUND/zh_val/val.json"
image_dir = "train_data/XFUND/zh_val/image"
save_dir = "train_data/XFUND/zh_val/label"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
with open(label_file, "r") as f:
    lines = f.readlines()
    for line in lines:
        image_name, ocr_infos = line.strip().split("\t")
        image_path = os.path.join(image_dir, image_name)
        image = cv2.imread(image_path)
        ocr_infos = json.loads(ocr_infos)
        canvas = draw_labels(ocr_infos, image)
        save_path = os.path.join(save_dir, image_name)
        # canvas.save(save_path)
"""
(head, tail) -> (question, answer)
一个 tail 对应一个 head
一个 head 对应多个 tail

"""


Corrupt JPEG data: premature end of data segment


In [47]:
my_relations = sorted(list(set(my_relations)))
from collections import defaultdict
groups = defaultdict(list)
for head, tail in my_relations:
    groups[tail].append(head)

for tail, heads in groups.items():
    if len(heads) > 1:
        print(tail, heads)